In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [4]:
def capm_1_asset(asset_returns: pd.Series, market_returns: pd.Series, risk_free_rate):
    """
    Calculate the CAPM alpha and beta for a given asset and market returns.
    CAPM model: Asset_excess_return (y) = Alpha (intercept) + Beta (x) * Market_premium (slope)
    Parameters:
    - asset_returns (pd.Series): The asset return series -> y
    - market_returns (pd.Series): The market return series -> x variable
    - risk_free_rate (float): The risk-free rate
    
    Returns:
    - dict: A dictionary containing alpha, beta, and the regression model summary.
    """
    # Step 1: Calculate Excess Returns
    asset_excess_returns = asset_returns - risk_free_rate
    market_excess_returns = market_returns - risk_free_rate

    # Step 2: Align the Data
    data = pd.DataFrame({
        'Asset Excess Return': asset_excess_returns,
        'Market Excess Return': market_excess_returns }).dropna()
    
    # Step 3: Add Constant to Market Returns for Intercept (Alpha)
    X = sm.add_constant(data['Market Excess Return'])  # Adds a constant term to the predictor
    
    # Step 4: Run the Regression
    model = sm.OLS(data['Asset Excess Return'], X).fit()
    
    # Step 5: Extract Alpha (intercept) and Beta (slope)
    alpha = model.params['const']
    beta = model.params['Market Excess Return']
    
    # Return the results as a dictionary
    return {
        'alpha': alpha,
        'beta': beta,
        'model_summary': model.summary()}

In [6]:
# Run CAPM loop for multiple assets
def capm_mul_assets_full(assets_return: pd.DataFrame, market_return: pd.Series, risk_free):
    # if needed, combine the Market data into the DF assets_return and we simply get market beta = 1
    results = {}
    for col in assets_return.columns:
    # Calculate CAPM alpha and beta for each column and store it in the results dictionary
        results[col] = capm_1_asset(assets_return[col], market_returns = market_return, risk_free_rate =  risk_free)
    return results

# Selective visualization the Alpha and Beta of multiple-asset CAPM as a DataFrame
def capm_mul_assets_coef(results): # the results is the variable assigned to capm_mul_assets_full

    # Initialize lists to store extracted alpha and beta values
    alpha_values = []
    beta_values = []
    
    # Iterate through the results dictionary to extract alpha and beta values
    for col, result in results.items():
        alpha_values.append((col, result['alpha']))  # Add (column name, alpha) tuple to list
        beta_values.append((col, result['beta']))    # Add (column name, beta) tuple to list
        
    # Convert the extracted values into DataFrames
    alpha_df = pd.DataFrame(alpha_values, columns=['Assets', 'ALPHA']).set_index('Assets')
    beta_df = pd.DataFrame(beta_values, columns=['Assets', 'BETA']).set_index('Assets')
    
    # Combine alpha and beta DataFrames into a single DataFrame
    alpha_beta_df = pd.concat([alpha_df, beta_df], axis=1)

    return alpha_beta_df

In [8]:
# SML CALCULATION
def sml_calc(df, risk_free_rate):
    """
    Calculate the SML regression equation using excess returns.
    SML quation: Asset_excess_return (y) = Beta (x) * Market_premium (slope)
    (In theory, intercept = 0)
    OR if we pre-add 2 sides by risk_free: Asset_return = Risk_free +  Beta (x) * Market_premium (slope)

    Parameters:
    df (pd.DataFrame): DataFrame with columns 'Mean Return' and 'Beta'.
    risk_free_rate (float): The risk-free rate.
    
    Returns:
    tuple: (intercept, slope, model) where:
        - intercept: Estimated risk-free rate (alpha)
        - slope: Estimated market risk premium
        - model: The regression model object
    """
    # Calculate excess returns for each asset
    df['Excess Return'] = df['Mean Return'] - risk_free_rate
    
    # Independent variable: Beta (X)
    X = df['Beta']
    X = sm.add_constant(X)  # Add a constant term for the intercept
    
    # Dependent variable: Excess Return (y)
    y = df['Excess Return']
    
    # Perform the regression
    model = sm.OLS(y, X).fit()
    
    # Extract the intercept (risk-free rate approximation) and slope (market risk premium)
    intercept = model.params['const'] + risk_free_rate  # Add back risk_free to display the SML
    slope = model.params['Beta']
    
    # Print the model summary for detailed statistics
    print(model.summary())
    
    return round(intercept, 4), round(slope, 4)

In [10]:
# SML CALCULATION AND PLOT
def sml_calc_and_plot(df, risk_free_rate, beta_range):
    """
    Calculate the SML regression equation using excess returns and plot the SML,
    market portfolio, and other assets.

    SML equation: Asset_excess_return (y) = Beta (x) * Market_premium (slope)
    (In theory, intercept = 0)
    OR if we pre-add 2 sides by risk_free: Asset_return = Risk_free +  Beta (x) * Market_premium (slope)

    Parameters:
    df (pd.DataFrame): DataFrame with columns 'Mean Return' and 'Beta'.
    risk_free_rate (float): The risk-free rate.
    """
    # Calculate excess returns for each asset
    df['Excess Return'] = df['Mean Return'] - risk_free_rate
    
    # Independent variable: Beta (X)
    X = df['Beta']
    X = sm.add_constant(X)  # Add a constant term for the intercept
    
    # Dependent variable: Excess Return (y)
    y = df['Excess Return']
    
    # Perform the regression
    model = sm.OLS(y, X).fit()
    
    # Extract the intercept (risk-free rate approximation) and slope (market risk premium)
    intercept = model.params['const'] + risk_free_rate  # Add back risk_free to display the SML
    slope = model.params['Beta']
    
    # Define the range for beta values to plot the SML
    beta_range = beta_range
    sml_values = intercept + slope * beta_range
    
    # Create the plot
    plt.figure(figsize=(15, 7))
    
    # Plot the SML line
    plt.plot(beta_range, sml_values, label='Security Market Line (SML)', color='olive')
    
    # Plot each asset
    for idx, row in df.iterrows():
        if idx == 'Market':
            # Plot the market portfolio in red
            plt.scatter(row['Beta'], row['Mean Return'], color='red', label='Market Portfolio', s= 70)
        else:
            # Plot other assets (industries) in blue
            plt.scatter(row['Beta'], row['Mean Return'], color='blue', s=50)
    
    # Add labels and legend
    plt.xlabel('Beta', fontsize = 14, color = 'navy')
    plt.ylabel('Expected Return', fontsize = 14, color = 'navy')
    plt.title('Security Market Line (SML) and Asset Positions', fontsize = 16, color = 'darkred')
    plt.legend(loc =  'best', fontsize = 15)
    plt.grid(True)
    
    # Show the plot
    plt.show()